<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 데이터 모델을 이용한 출력구조화 예제



In [ ]:
# 1) LangChain-OpenAI, LangChain-Core, Pydantic, OpenAI SDK 설치 (v0.3.0 이상)
!pip install --quiet --upgrade "langchain-openai>=0.3.0" "langchain-core>=0.3.0" pydantic openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 13.3 MB/s eta 0:00:00


In [ ]:
# 2) Colab Secrets에서 API 키 로드
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 필수 모듈 임포트
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

# 4) Pydantic 모델 정의
class CelestialBody(BaseModel):
    name: str = Field(description="천체의 이름(한글 표기)")
    radius: float = Field(description="천체 반경 (km)")
    mass: float = Field(description="천체 질량 (kg)")
    type: Literal["항성", "행성", "위성"] = Field(description="천체 종류")

# 5) LLM 인스턴스 생성 및 구조화된 출력 설정
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4.1-nano",  # 필요에 따라 모델명 조정
)
llm_structured = llm.with_structured_output(CelestialBody)

# 6) 호출 및 결과 획득
jupiter: CelestialBody = llm_structured.invoke("목성의 속성 정보를 알려주세요.")

# 7) 출력
print(f"목성 이름: {jupiter.name}")
print(f"목성 반경: {jupiter.radius} km")
print(f"목성 질량: {jupiter.mass} kg")
print(f"목성 유형: {jupiter.type}")


목성 이름: 목성
목성 반경: 69911.0 km
목성 질량: 1.898e+27 kg
목성 유형: 행성


# 중첩 데이터 모델을 이용한 출력구조

In [ ]:
# 1) LangChain-OpenAI, LangChain-Core, Pydantic, OpenAI SDK 설치 (v0.3.0 이상)
!pip install --quiet --upgrade "langchain-openai>=0.3.0" "langchain-core>=0.3.0" pydantic openai

In [ ]:
# 2) Colab Secrets에서 API 키 로드
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 필요한 모듈 임포트
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 4) Pydantic 모델 정의
class CelestialBody(BaseModel):
    name: str   = Field(description="천체 이름")
    radius: float = Field(description="천체 반경（km）")
    mass: float   = Field(description="천체의 질량（kg）")
    type: str     = Field(description="천체종류（행성, 항성, 위성 등）")

class PlanetarySystem(BaseModel):
    planets: List[CelestialBody]    = Field(description="행성 목록")
    center_body: CelestialBody      = Field(description="중심이 되는 항성")
    age: float                      = Field(description="행성계 연령 (억 년 단위 등)")
    name: str                       = Field(description="행성계 이름")

# 5) LLM 초기화 및 구조화된 출력 설정
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
llm_structured = llm.with_structured_output(PlanetarySystem)

# 6) 호출 및 결과 획득
solar_system: PlanetarySystem = llm_structured.invoke("태양계 정보를 알려주세요")

# 7) 결과 출력
print(f"시스템명: {solar_system.name}")
print(f"중심천체: {solar_system.center_body.name}")
print(f"행성계 연령: {solar_system.age}")
print("행성 목록:")
for planet in solar_system.planets:
    print(f"- {planet.name}")
    print(f"    종류: {planet.type}")
    print(f"    반경: {planet.radius} km")
    print(f"    질량: {planet.mass} kg")

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1669: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


시스템명: 태양계
중심천체: 태양
행성계 연령: 4567.0
행성 목록:
- 수성
    종류: 행성
    반경: 2439.7 km
    질량: 3.3011e+23 kg
- 금성
    종류: 행성
    반경: 6051.8 km
    질량: 4.8675e+24 kg
- 지구
    종류: 행성
    반경: 6371.0 km
    질량: 5.97237e+24 kg
- 화성
    종류: 행성
    반경: 3389.5 km
    질량: 6.4171e+23 kg
- 목성
    종류: 행성
    반경: 69911.0 km
    질량: 1.8982e+27 kg
- 토성
    종류: 행성
    반경: 58232.0 km
    질량: 5.6834e+26 kg
- 천왕성
    종류: 행성
    반경: 25362.0 km
    질량: 8.681e+25 kg
- 해왕성
    종류: 행성
    반경: 24622.0 km
    질량: 1.02413e+26 kg
